<div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 40px; margin-top: 0;">
    <div style="flex: 0 0 auto; margin-left: 0; margin-bottom: 0; margin-top: 0;">
        <img src="./pics/UCSD Logo.png" alt="UCSD Logo" style="width: 179px; margin-bottom: 0px; margin-top: 20px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/sdsc-logo.png" alt="SDSC Logo" style="width: 300px; margin-bottom: 0px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/usda-logo.png" alt="USDA Logo" style="width: 100px; margin-bottom: 0px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/wstc-logo.png" alt="WSTC Logo" style="width: 100px; height: 100px; margin-bottom: 0px;">
    </div>
</div>

<h1 style="text-align: center; font-size: 48px; margin-top: 0;">Rangeland Analysis Platform</h1>

In this notebook, we will explore the [Rangeland Analysis Platform (RAP)](https://rangelands.app/), the first in a series of introductory notebooks.

## What is RAP?

The Rangeland Analysis Platform (RAP) is a rangeland monitoring product developed by the United States Department of Agriculture. It provides a suite of spatial layers across the continental United States, primarily accessed through an online [interactive tool](https://rangelands.app/rap/?landcover_t=pfg&ll=39.0000,-98.0000&z=5).

Some of the key layers include:

- Perennial forb and grass cover  
- **Shrub cover**  
- Tree cover  
- Bare ground cover  

RAP data are available at 10 m and 30 m spatial resolution and are produced using the [Harmonized Landsat and Sentinel-2](https://hls.gsfc.nasa.gov/) surface reflectance products.

In this notebook, we will focus on RAP data published through the [Google Earth Engine Community Catalog](https://gee-community-catalog.org/projects/rap/). In particular, we will work with the shrub cover layer, which will serve as one of the core label sources in your pipeline.

# Importing our libraries

In [ ]:
import ee
import geemap
import folium
import numpy as np

## Google Earth Engine

To access RAP data, you will first need to authenticate with the Google Earth Engine (GEE) platform. This requires registering both your account and a project:

1. Go to the [Google Earth Engine site](https://earthengine.google.com/).
2. In the top-right corner, click **Get Started**.
3. Log in using your **institutional account**. This is important because Google Earth Engine offers free access for academic use.
4. Click on **Register a new project**.
5. Name your project: `shrubwise-dc`.
6. Fill out the project registration form as follows:
   - **Organization**: Public or private academic institution  
   - **Institution**: Enter the name of your institution  
   - Indicate that you **will not receive any payment** from this project  
   - Indicate that you will use Earth Engine for **Scientific Research**, and specify your research question as **Shrubland coverage**  
   - For **Geographic scope**, select **Regional** and specify **California** as the region  
   - For **Category**, select **Mitigation**  
   - Indicate that you will use Earth Engine for **Other, Natural Disasters / Climate Risk, Forestry**, and use **wildfires** as the short description
7. Submit the form and wait for your project to be approved and credentials generated.

Once your credentials are available, you will use them in the next cell to authenticate your Earth Engine session.

In [ ]:
ee.Authenticate()
ee.Initialize(project='shrubwise-dc') # Replace with your project's name

## Accessing the Data
The first thing we will do, will be accessing the data. We will access the vegetation cover at a 10m resolution. As we mentioned previously, we're using the data published through the [Google Earth Community Catalog](https://gee-community-catalog.org/projects/rap/).

In [ ]:
RAP_veg_yearly_10m = ee.ImageCollection('projects/rap-data-365417/assets/vegetation-cover-10m')

Let's take a look at the total number of images at the 10m collection.

In [ ]:
print("Total images in collection:", RAP_veg_yearly_10m.size().getInfo())
first_img = RAP_veg_yearly_10m.first()
print("\nFirst image info (short):")
print(first_img.getInfo()['properties'].keys())

#### List of available years

In [ ]:
years = (RAP_veg_yearly_10m
         .aggregate_array('year')
         .distinct()
         .sort()
         .getInfo())

print("Available years:", years)


#### Band names

In [ ]:
band_names = first_img.bandNames().getInfo()
print("Band names:", band_names)

We are going to focus on the `SHR` band, as it corresponds to the shrub coverage.

#### 2020 Example

In [ ]:
year = 2020

images_2020 = (RAP_veg_yearly_10m
               .filter(ee.Filter.eq('year', year)))

print("Number of images for", year, ":", images_2020.size().getInfo())

# Inspect the IDs so you see what's there - Uncomment the print
ids_2020 = images_2020.aggregate_array('system:index').getInfo()
#print("Image IDs for", year, ":\n", ids_2020)

# Turn collection into a list and pick (Here we use index 35 because this image lands in California)
images_list = images_2020.toList(images_2020.size())
img_2020_alt = ee.Image(images_list.get(35))   

print("Chosen image ID:", img_2020_alt.get('system:index').getInfo())

# We specifically select the SHR band and rename it as shrub_cover
shrub_2020 = img_2020_alt.select('SHR').rename('shrub_cover')

#### General Properties

Now, let's inspect at the available information for the selected tile. 

In [ ]:
info = shrub_2020.getInfo()

print("Top-level keys:", info.keys())       

print("\nBands:")
for b in info['bands']:
    print(b)

print("\nProperties:")
for k, v in info['properties'].items():
    print(f"{k}: {v}")

One of the main pieces of information to notice here is that each image is composed by 7550x7550 pixels, where each pixel is at a 10m resolution.

In [ ]:
# Band names (should just be functional groups, including SHR)
print("Bands:", img_2020_alt.bandNames().getInfo())

# All properties (where vis metadata would live, if present)
props = img_2020_alt.propertyNames().getInfo()
print("\nProperties:")
for p in props:
    print(p, "=", img_2020_alt.get(p).getInfo())

#### Visualization

Now, let's visualize the image that we selected.

In [ ]:
# Visualization parameters
vis_params = {
    'min': 0,
    'max': 100,  # percent cover
    'palette': ['ffffff', 'c7e9c0', '74c476', '238b45', '00441b']
}

# Get map tile URL from EE
map_dict = shrub_2020.getMapId(vis_params)

m = folium.Map(location=[39, -120], zoom_start=6)

folium.TileLayer(
    tiles=map_dict['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='Shrub cover 2020'
).add_to(m)

folium.LayerControl().add_to(m)

m 

If you zoom in to the northwest of Sacramento, you will see the overlapped image corresponding to the shrub coverage for that area.

### Array Representation

The map we just displayed is a great way to see the RAP shrub cover layer, but it isn’t directly usable for machine learning. For training, we need the underlying data as numbers: shrub cover values at the pixel level.

In this section, we’ll extract a sample rectangle from a region of interest (ROI) and convert it into a 2D array. In that array, each cell corresponds to a 10 m × 10 m pixel, and the value is the estimated shrub cover (%) for that pixel.

We use an ROI because Earth Engine’s `sampleRectangle()` has a practical limit of about 262,144 pixels per request, while a full RAP 10 m tile contains ~57 million pixels.

A key part of the challenge is scaling this approach to the full tile: you’ll eventually need to extract all pixels, but you’ll do it chunk by chunk. That means designing a tiling/chunking strategy where each ROI covers a unique area—no overlaps and no gaps—so every pixel is collected exactly once.

In [ ]:
import numpy as np

# Tile bounds from the tile
min_lon = -123.0029673024087
max_lon = -122.12443886634831
min_lat =  38.8433024023722
max_lat =  39.52694585089132

km = 5.0

# Convert km to degrees (approx).
dlat = km / 111.32
dlon = km / (111.32 * np.cos(np.deg2rad(min_lat)))

# Build a SOUTH-WEST (lower-left) corner ROI that extends inward 
roi = ee.Geometry.Rectangle(
    [min_lon, min_lat, min_lon + dlon, min_lat + dlat],
    geodesic=False
)

# Extract array
img = shrub_2020.toFloat()
rect = img.sampleRectangle(region=roi, defaultValue=-9999)

arr = np.array(rect.get('shrub_cover').getInfo(), dtype=np.float32)
arr[arr == -9999] = np.nan

print("array:", arr.shape, arr.dtype)
print("min/max:", np.nanmin(arr), np.nanmax(arr))

Success! We have a region of interest represented as an array. As you will notice, no pixel will have a value below 0 or above 100, since each value represents the % of shrub coverage.

Let's see our array.

In [ ]:
arr

### Task: Explore another region in California 

Proceed to the `tasks.ipynb` notebook and complete the tasks associated to this workspace.